# SSMR: generate SSM model from trajectory data

In [1]:
import numpy as np
from scipy.io import loadmat
from copy import deepcopy
import os

In [2]:
%matplotlib qt
import matplotlib.pyplot as plt

In [3]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

In [4]:
np.set_printoptions(linewidth=300)

In [5]:
use_data = "mine" # "mine", "john"

In [10]:
if use_data == "mine":
    diamond_dir = "../../../soft-robot-control/examples/diamond"
    decayData_dir = os.path.join(diamond_dir, "dataCollection/origin")
    rest_file = os.path.join(diamond_dir, 'rest.pkl')

    output_node = 1354
    t_in = 2
    t_out = 5

    oData = utils.import_pos_data(decayData_dir, rest_file, output_node, t_in, t_out)

## Import and inspect data
See mat file in URL

In [11]:
dataDecayObs = loadmat("../dataDecayObs.mat") # , simplify_cells=True)

Data = {}
if use_data == "john":
    Data['oData'] = dataDecayObs['oData'].tolist()
    # Data['yData'] = dataDecayObs['yData'].tolist()
    # Xsnapshots = dataDecayObs['Xsnapshots']
    for i in range(len(Data['oData'])):
        Data['oData'][i][0] = Data['oData'][i][0].squeeze()
        # Data['yData'][i][0] = Data['yData'][i][0].squeeze()
else: # use my data
    Data['oData'] = oData

rDOF = 3
oDOF = 3
nTRAJ = len(Data['oData'])
SSMDim = 6

print("nTRAJ:", nTRAJ)

nTRAJ: 80


### Plot trajectories in an observable of interest

In [13]:
outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'])
# plot evolution of x, y and z in time, separately in 3 subplots
highlight_traj = [] # [48, 49, 50, 51, 54, 55, 56, 57, 58, 59]
plot.traj_xyz(Data,
             xyz_idx=[('oData', 0), ('oData', 1), ('oData', 2)],
             xyz_names=[r'$x$', r'$y$', r'$z$'],
             highlight_idx=highlight_traj)
# plot trajectories in 3D [x, x_dot, z] space / NB: x_dot = v_x
# plot.traj_3D(Data,
#              xyz_idx=[('oData', 0), ('oData', 0+oDOF), ('oData', 2)],
#              xyz_names=[r'$x$', r'$\dot{x}$', r'$z$'])

Downsample data to $dt = 0.01$

In [14]:
if use_data == 'john':
    for i in range(nTRAJ):
        Data['oData'][i][0] = Data['oData'][i][0][::10]
        Data['oData'][i][1] = Data['oData'][i][1][:, ::10]

Compute reduced coordinates using delay embedding on the available oData

In [15]:
N_DELAY = 4

In [16]:
Data['yData'] = deepcopy(Data['oData'])
for i in range(nTRAJ):
    delay_embedded_traj = utils.delayEmbedding(Data['oData'][i][1], up_to_delay=N_DELAY)
    Data['yData'][i][1] = delay_embedded_traj[:, :-4]
    Data['yData'][i][0] = Data['yData'][i][0][4:]

In [19]:
t_interval = [0.08, 2.6]
Data['oDataTrunc'] = utils.slice_trajectories(Data['oData'], t_interval)
Data['yDataTrunc'] = utils.slice_trajectories(Data['yData'], t_interval)

In [20]:
# print(Data['yDataTrunc'][0][1].shape)
# Data['yDataTrunc'][0][1][:, :5]

## Obtain reduced-order coordinates

### Perform PCA on delay-embedded coordinates to obtain reduced coordinates

In [21]:
# Perform SVD on displacement field
# if use_data == "mine":
from scipy.sparse.linalg import svds

Xsnapshots = np.hstack([yDataTrunc[1] for yDataTrunc in Data['yDataTrunc']])
# subsample dt=0.01
Xsnapshots = Xsnapshots[:, :]

# print(Xsnapshots.shape)
# print(Xsnapshots[:, 0:10])

v, s, u = svds(Xsnapshots, k=SSMDim, which="LM")
ind = np.argsort(s)[::-1]
s = s[ind]
v = v[:, ind]
# v, s, _ = np.linalg.svd(Xsnapshots, compute_uv=True)
# print(s)
# print(v)
l2vals = s**2

In [23]:
# Plot variance description: we expect three modes to capture almost all variance.
# Note we assume data centered around the origin, which is the fixed point of our system.
plt.close('all')
plot.pca_modes(l2vals, up_to_mode=9)

### Project delay-embedded data onto dominant modes

In [24]:
from copy import deepcopy
Data['etaDataTrunc'] = deepcopy(Data['yDataTrunc'])
Vde = v[:, :SSMDim]
for i in range(nTRAJ):
    Data['etaDataTrunc'][i][1] = Vde.T @ Data['yDataTrunc'][i][1]

In [25]:
plt.close('all')
# plot first three reduced coordinates
plot.traj_xyz(Data,
              xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 1), ('etaDataTrunc', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'])

### Train and test data (train/test split)

In [26]:
indTest = [0, 4] # [0, 2, 4, 6, 8, 10, 12, 14]
indTrain = [i for i in range(nTRAJ) if i not in indTest]

## SSMLearn

Start Matlab engine and install/run SSMLearn

In [27]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.cd("../..", nargout=0)
eng.install(nargout=0)
eng.cd("ROM/")

SSMLearn: Data-driven Reduced Order Models for Nonlinear Dynamical Systems
Maintained by Mattia Cenedese (mattiac@ethz.ch) and Joar Axs (jgoeransson@ethz.ch)
License: GNUv3.0
> In path (line 109)
In addpath (line 86)
In install (line 20)


'/home/jonas/Projects/stanford/SSMR-for-control'

Bring data in a format that is accepted by the (slightly modified) version of SSMLearn (requires a specific cell array structure for input data)

In [28]:
# make data ready for Matlab
yDataTruncTrain_matlab = [[], []]
etaDataTruncTrain_matlab = [[], []]
for i in indTrain:
    yDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['yDataTrunc'][i][0].tolist()))
    yDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['yDataTrunc'][i][1].tolist()))
    etaDataTruncTrain_matlab[0].append(matlab.double(initializer=Data['etaDataTrunc'][i][0].tolist()))
    etaDataTruncTrain_matlab[1].append(matlab.double(initializer=Data['etaDataTrunc'][i][1].tolist()))


### Learn geometry of the SSM

Find parametrization of SSM using SSMLearn

In [29]:
SSMOrder = 3

print(Data['etaDataTrunc'][0][1].shape)

IMInfo = eng.IMGeometry(yDataTruncTrain_matlab, SSMDim, SSMOrder,
                        'reducedCoordinates', etaDataTruncTrain_matlab)
# IMInfoInv = eng.IMGeometry(yDataTruncTrain_matlab, obsDim, SSMOrder,
#                            'reducedCoordinates', oDataTruncTrain_matlab)


(6, 253)
Data seems to have been fed from Python!


### Learn dynamics on the SSM (reduced dynamics)

Find parametrization of reduced dynamics using SSMLearn

In [30]:
ROMOrder = 3
RDInfo = eng.IMDynamicsFlow(etaDataTruncTrain_matlab, 'R_PolyOrd', ROMOrder, 'style', 'default')

Data seems to have been fed from Python!
Estimation of the reduced dynamics... 
 Done. 


Stop Matlab engine -- not needed anymore

In [31]:
eng.quit()

Convert all matlab double arrays to numpy arrays

In [32]:
def convert_matlab_arrays_to_numpy_arrays(info_dict):
  for k,v in info_dict.items():        
     if isinstance(v, dict):
         convert_matlab_arrays_to_numpy_arrays(v)
     else:            
        if isinstance(v, matlab.double):
           info_dict[k] = np.array(v)

convert_matlab_arrays_to_numpy_arrays(IMInfo)
convert_matlab_arrays_to_numpy_arrays(RDInfo)

## Analyze the obtained mappings of SSM geometry and reduced dynamics

### SSM geometry (parametrization)

In [33]:
yRec = {}
errorGeo = {}
meanErrorGeo = {}

#### Train error

In [34]:
yRec['Train'] = utils.lift_trajectories(IMInfo, [Data['etaDataTrunc'][i] for i in indTrain])
errorGeo['Train'] = utils.compute_trajectory_errors(yRec['Train'], [Data['yDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGeo['Train'] = np.mean(errorGeo['Train'])
print(f"Average parametrization train error: {meanErrorGeo['Train']:.4e}")

Average parametrization train error: 2.4568e-03


#### Test error

In [35]:
yRec['Test'] = utils.lift_trajectories(IMInfo, [Data['etaDataTrunc'][i] for i in indTest])
errorGeo['Test'] = utils.compute_trajectory_errors(yRec['Test'], [Data['yDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGeo['Test'] = np.mean(errorGeo['Test'])
print(f"Average parametrization test error: {meanErrorGeo['Test']:.4e}")

Average parametrization test error: 2.8593e-03


Plot comparison of SSM-predicted vs. actual test trajectories

In [36]:
print(Data['yData'][0][1].shape)
print(yRec['Test'][0][1].shape)

(15, 296)
(15, 253)


In [37]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('Test', 0), ('Test', 1), ('Test', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

### Reduced dynamics

In [38]:
etaRec = {}
etaRec['RD'] = utils.advectRD(RDInfo, Data['etaDataTrunc'])[0]
meanErrorDyn = {}

for i in range(nTRAJ):
    if etaRec['RD'][i][1].shape[1] != Data['etaDataTrunc'][i][1].shape[1]:
        print(i)

normedTrajDist = utils.compute_trajectory_errors(etaRec['RD'], Data['etaDataTrunc'])[0]
meanErrorDyn['Train'] = np.mean(normedTrajDist[indTrain]) * 100
meanErrorDyn['Test'] = np.mean(normedTrajDist[indTest]) * 100

print(f"Average dynamics train error: {meanErrorDyn['Train']:.4f}")
print(f"Average dynamics test error: {meanErrorDyn['Test']:.4f}")

Average dynamics train error: 0.1114
Average dynamics test error: 0.1317


In [39]:
plot_all = False
if plot_all:
    vPlot = range(nTRAJ)
else:
    vPlot = indTest

plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 1), ('etaDataTrunc', 2)],
                    xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
                    traj_idx=vPlot,
                    show=False)
plot.traj_xyz(etaRec,
              xyz_idx=[('RD', 0), ('RD', 1), ('RD', 2)],
              xyz_names=[r'$u_1$', r'$u_2$', r'$u_3$'],
              traj_idx=vPlot,
              axs=axs, ls=':', color='darkblue')


### Global error

In [40]:
errorGlo = {}
meanErrorGlo = {}

#### Train error

In [41]:
yRec['RDTrain'] = utils.lift_trajectories(IMInfo, [etaRec['RD'][i] for i in indTrain])
errorGlo['Train'] = utils.compute_trajectory_errors(yRec['RDTrain'], [Data['yDataTrunc'][i] for i in indTrain])[0] * 100

meanErrorGlo['Train'] = np.mean(errorGlo['Train'])
print(f"Average global train error: {meanErrorGlo['Train']:.4f}")

Average global train error: 0.1117


#### Test error

In [42]:
yRec['RDTest'] = utils.lift_trajectories(IMInfo, [etaRec['RD'][i] for i in indTest])
errorGlo['Test'] = utils.compute_trajectory_errors(yRec['RDTest'], [Data['yDataTrunc'][i] for i in indTest])[0] * 100

meanErrorGlo['Test'] = np.mean(errorGlo['Test'])
print(f"Average global test error: {meanErrorGlo['Test']:.4f}")

Average global test error: 0.1322


In [43]:
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(yRec,
              xyz_idx=[('RDTest', 0), ('RDTest', 1), ('RDTest', 2)],
              xyz_names=[r'$x$', r'$y$', r'$z$'],
              axs=axs, ls=':', color='darkblue')

## Control

### Setup model for control

In [44]:
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d 

In [45]:
Rauton = utils.Rauton(RDInfo)
Vauton = lambda x: IMInfo['parametrization']['H'] @ utils.multivariate_polynomial(x, IMInfo['parametrization']['polynomialOrder'])
Wauton = lambda y: Vde.T @ y

### Learn control matrix $B$

Use figure8 trajectory as the training data

In [46]:
# Load the controls and actual trajectories for figure8
u_train = np.loadtxt("../../system_data/u_big.csv", delimiter=',').T
z_train = np.loadtxt("../../system_data/z_big.csv", delimiter=',').T
z_eq_train = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
print(z_eq_train)

# tip position, normalized using resting position
z_shift_train = (z_train[-3:, :].T - z_eq_train).T

# original spacing of inputs
T = 10.01
dt = 0.01
N = int(T / dt)
t_train = np.linspace(0, T, N+1)

[-7.07686434  1.2132194  97.04053866]


Use randomly sampled (step) inputs as the training data

In [47]:
diamond_train_full = loadmat("../diamond_train_full.mat", simplify_cells=True)
z_train, u_train, _ = diamond_train_full['y'].T, diamond_train_full['u'].T, diamond_train_full['t']
print(z_train.shape)

# tip position, normalized using resting position
z_shift_train = (z_train[-3:, :].T - z_train[-3:, 0]).T
dt = 0.01
t_train = np.arange(z_train.shape[1]) * dt

(6, 8692)


Plot the training data

In [48]:
plt.close('all')
plot.traj_xyz_txyz(t=t_train,
                   x=z_shift_train[0, :], y=z_shift_train[1, :], z=z_shift_train[2, :])
plot.inputs(t_train, u_train)

Do delay embedding on the tip position (and optionally, on the inputs). Then, compute reduced coordinates and gradient as predicted by reduced dynamics, as well as the numerical (true) gradient

In [49]:
# delay-embed tip position (as well as inputs?)
y_train = utils.delayEmbedding(z_shift_train, embed_coords=[0, 1, 2], up_to_delay=N_DELAY)
print(f"y.shape: {y_train.shape}, u.shape: {u_train.shape}")
# reduced coordinates
print(t_train[:5])
x_train = Wauton(y_train)
print(x_train[0, :5])
dxDt = np.gradient(x_train, dt, axis=1)
dxDt_ROM = Rauton(x_train)

# optionally, throw away some of the training data
# keep_indices = np.s_[4000:-100]
keep_indices = np.s_[30:-20]
t_train_trunc = t_train[keep_indices]
u_train_trunc = u_train[:, keep_indices]
x_train_trunc = x_train[:, keep_indices]
y_train_trunc = y_train[:, keep_indices]
dxDt = dxDt[:, keep_indices]
dxDt_ROM = dxDt_ROM[:, keep_indices]

print(t_train.shape)

y.shape: (15, 8692), u.shape: (4, 8692)
[0.   0.01 0.02 0.03 0.04]
[0.10504803 0.1624417  0.22091606 0.28590054 0.37056534]
(8692,)


In [50]:
plt.close('all')
ax = plot.traj_3D_xyz(y_train_trunc[outdofs[0], :],
                      y_train_trunc[outdofs[1], :],
                      y_train_trunc[outdofs[2], :], show=True)

Plot gradients (numerical differentiation and reduced dynamics)

In [51]:
print(dxDt.shape)
plot.reduced_coordinates_gradient(t_train_trunc, [dxDt, dxDt_ROM], labels=["true numerical", "predicted autonomous"], how="all")

(6, 8642)


In [52]:
plt.close('all')
plot.dependence_of_xdot_on_inputs(dxDt, u_train_trunc)

Learn $B$ matrix

In [53]:
from sklearn.linear_model import Ridge

polyUorder = 1
# assemble_features = lambda u, x: utils.multivariate_polynomial(np.vstack([u, x]), order=polyUorder)
assemble_features = lambda u, x: utils.multivariate_polynomial(u, order=polyUorder)

X = assemble_features(u_train_trunc, x_train_trunc)
print(X.shape)

ridgeModel = Ridge(alpha=1000, fit_intercept=False)
ridgeModel.fit(X.T, (dxDt - dxDt_ROM).T)
B_learn = ridgeModel.coef_
print(B_learn.shape)

(4, 8642)
(6, 4)


Frobenius norm of $B_{learn}$ to estimate how much influence inputs have on dynamics

In [54]:
print(f"Frobenius norm of B_learn: {np.linalg.norm(B_learn, ord='fro'):.4f}")

Frobenius norm of B_learn: 0.3200


Plot gradients again, but this time, using the learned influence of inputs for the prediction

In [55]:
plt.close('all')
plot_reduced_coords = np.s_[:] # [3, 4, 5]
dxDt_ROM_with_B = Rauton(x_train_trunc) + B_learn @ assemble_features(u_train_trunc, x_train_trunc) # utils.multivariate_polynomial(u_train_trunc, order=polyUorder)
plot.reduced_coordinates_gradient(t_train_trunc, [dxDt[plot_reduced_coords, :], dxDt_ROM[plot_reduced_coords, :], dxDt_ROM_with_B[plot_reduced_coords, :]], labels=["true numerical", "predicted autonomous", "predicted with inputs"], how="all")

In [56]:
# # home-brewed ridge regression
# alpha = 0
# X, y = U.T, (dXbarDt - dXbarDt_ROM).T
# B_learn = np.linalg.solve(X.T @ X + alpha * np.eye(X.shape[0]), X.T @ y)

### Integrate model with inputs (big amplitude) -- and using learned influence of control

Try to predict the training trajectory using reduced dynamics

In [57]:
# Load the controls and actual trajectories for figure8
u_test = np.loadtxt("../../system_data/u_big.csv", delimiter=',').T
z_test = np.loadtxt("../../system_data/z_big.csv", delimiter=',').T
z_eq_test = np.loadtxt("../../system_data/z_equilibrium.csv", delimiter=',')
print(z_test.shape)
# tip position, normalized using resting position
z_shift_test = (z_test[-3:, :].T - z_eq_test).T
# delay-embed tip position (as well as inputs?)
y_test = utils.delayEmbedding(z_shift_test, embed_coords=[0, 1, 2], up_to_delay=N_DELAY)
# reduced coordinates
x_test = Wauton(y_test)

# original spacing of inputs
T = 10.01
dt = 0.01
N = int(T / dt)
t_test = np.linspace(0, T, N+1)

slice_traj = np.s_[30:-20]
t_test = t_test[slice_traj]
u_test = u_test[:, slice_traj]
x_test = x_test[:, slice_traj]
z_test = z_test[:, slice_traj]

(6, 1002)


In [58]:
uInterpFun = interp1d(t_test, u_test, axis=1, fill_value="extrapolate")
uFun = lambda t: uInterpFun(t).reshape(-1, 1)
# print(t_test.shape, dxDt.shape)
# dxDtInterpFun = interp1d(t_test, dxDt_ROM, axis=1, fill_value="extrapolate")
def R(t, y):
    return Rauton(np.atleast_2d(y)) + B_learn @ assemble_features(uFun(t), y) # utils.multivariate_polynomial(uFun(t), order=polyUorder)
    # return dxDtInterpFun(t)

# solve IVP of reduced dynamics using open-loop figure8 inputs
sol = solve_ivp(R,
                t_span=[t_test[0], t_test[-1]],
                t_eval=t_test,
                y0=x_test[:, 0], # np.zeros(2*rDOF),
                method='RK45',
                vectorized=True,
                rtol=1e-3,
                atol=1e-3)
# resulting (predicted) open-loop trajectory in reduced coordinates
xTraj = sol.y
yTraj = Vauton(xTraj)
zTraj = (yTraj[:3, :].T + z_eq_test).T
print(zTraj.shape)

plt.close('all')
ax = plot.traj_3D_xyz(zTraj[outdofs[0], :],
                      zTraj[outdofs[1], :],
                      zTraj[outdofs[2], :], show=False)
ax = plot.traj_3D_xyz(z_test[3, :],
                      z_test[4, :],
                      z_test[5, :], color="tab:orange", ax=ax, show=False)
ax.legend(["Predicted trajectory", "Actual trajectory"])
plt.show()


(3, 952)


In [59]:
RMSE = np.sum(np.sqrt(np.mean((zTraj[0:3, :] - z_test[-3:])**2, axis=0))) / zTraj.shape[1]
print(f"RMSE = {RMSE:.4f}")

RMSE = 0.4962


## Save SSM model

Continuous model

In [60]:
SSM_model = {'model': {}, 'params': {}}

SSM_model['model']['w_coeff'] = IMInfo['parametrization']['H']
SSM_model['model']['v_coeff'] = None # Vde # IMInfo['chart']['H']
SSM_model['model']['r_coeff'] = RDInfo['reducedDynamics']['coefficients']
SSM_model['model']['B'] = B_learn
SSM_model['model']['V'] = Vde

SSM_model['params']['SSM_order'] = SSMOrder
SSM_model['params']['ROM_order'] = ROMOrder
SSM_model['params']['state_dim'] = 2 * rDOF
print(2*rDOF)

SSM_model['params']['input_dim'] = B_learn.shape[1]
SSM_model['params']['output_dim'] = 3 # 2 * oDOF
SSM_model['params']['delays'] = N_DELAY
SSM_model['params']['obs_dim'] = 3 * (1 + N_DELAY)

6


In [61]:
import pickle
# save dictionary to person_data.pkl file
with open('../../../soft-robot-control/examples/hardware/SSMmodels/SSM_model_delayEmbedding.pkl', 'wb') as f:
    pickle.dump(SSM_model, f)